<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/posts/02wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상 

# 2. Imports 

In [2]:
import datasets
import transformers
import evaluate
import numpy as np
import torch # 파이토치

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. 이전시간 요약

`-` 이전시간의 내용중 이번시간에 기억할것들을 요약 

`-` `DatasetDict`: 임의의 자료에 대한 `DatasetDict` 오브젝트 만들기

In [90]:
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}
train_data = datasets.Dataset.from_dict(train_dict)
test_data = datasets.Dataset.from_dict(test_dict)
나의데이터 = datasets.dataset_dict.DatasetDict({'train':train_data, 'test':test_data})

`-` `DatasetDict`: 아래의 코드도 가능 

In [92]:
나의데이터['train']['text']

['I prefer making decisions based on logic and objective facts.',
 'I always consider how others might feel when making a decision.',
 'Data and analysis drive most of my decisions.',
 'I rely on my empathy and personal values to guide my choices.']

`-` `토크나이저`: 토크나이저는 "`Str` $\to$ `Dict`" 인 함수이다.

In [98]:
토크나이저 = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 

In [101]:
토크나이저(나의데이터['train']['text'][0])

{'input_ids': [101, 1045, 9544, 2437, 6567, 2241, 2006, 7961, 1998, 7863, 8866, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

`-` `토크나이저`: 토크나이저의 "Str $\to$ Dict" 인 함수는 배치처리가 가능하다. 

In [102]:
토크나이저(나의데이터['train']['text'])

{'input_ids': [[101, 1045, 9544, 2437, 6567, 2241, 2006, 7961, 1998, 7863, 8866, 1012, 102], [101, 1045, 2467, 5136, 2129, 2500, 2453, 2514, 2043, 2437, 1037, 3247, 1012, 102], [101, 2951, 1998, 4106, 3298, 2087, 1997, 2026, 6567, 1012, 102], [101, 1045, 11160, 2006, 2026, 26452, 1998, 3167, 5300, 2000, 5009, 2026, 9804, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

`-` `인공지능`: 인공지능은 많은 파라메터를 포함하고 있는 어떠한 물체이다. 

`-` `인공지능`: 인공지능의 파라메터는 변화할 수 있으며, loss가 더 작은쪽으로 파라메터를 변화시키는 과정을 "학습"이라고 부른다. 

`-` `인공지능`: 인공지능은 "`**Dict` $\to$ `transformers.modeling_outputs.SequenceClassifierOutput`"인 함수이다. 그런데 쓰기 까다롭다.

- `1`. `Dict`에는 특정한 key를 포함하고 있어야한다. (`input_ids`, `attention_mask`) 
- `2`. key에 대응하는 숫자는 파이토치 텐서형태이어야 한다. (`3`. 따라서 (m,n)꼴의 차원을 **반드시** 가져야 한다)
- `4`. `Dict`에 `labels`이 (텐서형으로) 포함된 경우 loss가 계산된다. (그리고 이걸 계산해야지 학습을 할 수 있음) 

In [133]:
인공지능 = model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`#` 입력예시1

In [184]:
인공지능(
    **{
        'input_ids': torch.tensor([[ 101, 1045, 9544, 2437, 6567, 2241, 2006, 7961, 1998, 7863, 8866, 1012, 102]]), 
        'attention_mask': torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]) # 생략가능
    }
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1083,  0.0203]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#` 입력예시2

In [217]:
인공지능(
    **{
        'input_ids': torch.tensor([[ 101, 1045, 9544, 2437, 6567, 2241, 2006, 7961, 1998, 7863, 8866, 1012, 102]]), 
        'attention_mask': torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), # 생략가능
        'labels': torch.tensor([1]) # 생략가능
    }
)

SequenceClassifierOutput(loss=tensor(0.6309, grad_fn=<NllLossBackward0>), logits=tensor([[-0.1083,  0.0203]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`-` `인공지능`: 인공지능의 출력결과 "`transformers.modeling_outputs.SequenceClassifierOutput`" 에서 유의미한 정보를 추출하는 방법

- `1`. `Dict`에는 특정한 key를 포함하고 있어야한다. (`input_ids`, `attention_mask`) 
- `2`. key에 대응하는 숫자는 파이토치 텐서형태이어야 한다. (`3`. 따라서 (m,n)꼴의 차원을 **반드시** 가져야 한다)
- `4`. `Dict`에 `labels`이 (텐서형으로) 포함된 경우 loss가 계산된다. (그리고 이걸 계산해야지 학습을 할 수 있음) 

# 4. 데이터전처리하기2

`-` 아래코드를 파고들어보자. 

```Python
def 데이터전처리하기2(examples):
    return 데이터전처리하기1(examples["text"], truncation=True)
전처리된나의데이터 = 나의데이터.map(lambda x: {'dummy': '메롱'})
```

`-` `examples['text']`의 쓰임으로 유추해본결과 examples에 가정된 입력의 형태?? 

In [189]:
train_data['text']

['I prefer making decisions based on logic and objective facts.',
 'I always consider how others might feel when making a decision.',
 'Data and analysis drive most of my decisions.',
 'I rely on my empathy and personal values to guide my choices.']

In [190]:
train_data[0]['text']

'I prefer making decisions based on logic and objective facts.'

`-` 개념1: Hugging Face의 `datasets` 라이브러리에서 제공하는 `datasets.dataset_dict.DatasetDict`은, 요소들의 변환에 특화된 `map`이라는 메소드가(=함수가) 내장되어있다. 

`# 예시1`

In [14]:
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}
train_data = datasets.Dataset.from_dict(train_dict)
test_data = datasets.Dataset.from_dict(test_dict)
나의데이터 = datasets.dataset_dict.DatasetDict({'train':train_data, 'test':test_data})

In [64]:
datasets.dataset_dict.DatasetDict

datasets.dataset_dict.DatasetDict

In [ ]:
datasets.dataset_dict.DatasetDict

In [63]:
나의데이터.map?

Signature:
나의데이터.map(
    function: Optional[Callable] = None,
    with_indices: bool = False,
    with_rank: bool = False,
    input_columns: Union[str, List[str], NoneType] = None,
    batched: bool = False,
    batch_size: Optional[int] = 1000,
    drop_last_batch: bool = False,
    remove_columns: Union[str, List[str], NoneType] = None,
    keep_in_memory: bool = False,
    load_from_cache_file: Optional[bool] = None,
    cache_file_names: Optional[Dict[str, Optional[str]]] = None,
    writer_batch_size: Optional[int] = 1000,
    features: Optional[datasets.features.features.Features] = None,
    disable_nullable: bool = False,
    fn_kwargs: Optional[dict] = None,
    num_proc: Optional[int] = None,
    desc: Optional[str] = None,
) -> 'DatasetDict'
Docstring:
Apply a function to all the elements in the table (individually or in batches)
and update the table (if function does updated examples).
The transformation is applied to all the datasets of the dataset dictionary.

Args:
   

- 당장필요한내용: `datasets.dataset_dict.DatasetDict.map()`은 (1) `function`을 입력으로 받는데, (2) `function`은 Dict를 입력으로 받고 Dict를 출력하는 함수이어야 한다. 

In [60]:
전처리된나의데이터 = 나의데이터.map(lambda x: {'dummy': '메롱'})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1012.51 examples/s]


In [52]:
나의데이터['train'][0], 전처리된나의데이터['train'][0]

({'text': 'I prefer making decisions based on logic and objective facts.',
  'label': 0},
 {'text': 'I prefer making decisions based on logic and objective facts.',
  'label': 0,
  'dummy': '메롱'})

In [57]:
for d in 전처리된나의데이터['train']:
    print(d)

{'text': 'I prefer making decisions based on logic and objective facts.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': 'I always consider how others might feel when making a decision.', 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}
{'text': 'Data and analysis drive most of my decisions.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': 'I rely on my empathy and personal values to guide my choices.', 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}


In [58]:
for d in 전처리된나의데이터['test']:
    print(d)

{'text': 'I find it important to weigh all the pros and cons logically.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': "When making decisions, I prioritize harmony and people's emotions.", 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}


`#`

`# 예시2`

In [53]:
전처리된나의데이터 = 나의데이터.map(lambda x: {'dummy':'메롱', 'label2': '사고형(T)'} if x['label'] == 0 else {'dummy':'메롱','label2': '감정형(F)'})

Map: 100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 665.02 examples/s]


In [54]:
for d in 전처리된나의데이터['train']:
    print(d)

{'text': 'I prefer making decisions based on logic and objective facts.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': 'I always consider how others might feel when making a decision.', 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}
{'text': 'Data and analysis drive most of my decisions.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': 'I rely on my empathy and personal values to guide my choices.', 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}


In [55]:
for d in 전처리된나의데이터['test']:
    print(d)

{'text': 'I find it important to weigh all the pros and cons logically.', 'label': 0, 'dummy': '메롱', 'label2': '사고형(T)'}
{'text': "When making decisions, I prioritize harmony and people's emotions.", 'label': 1, 'dummy': '메롱', 'label2': '감정형(F)'}


`#`

`# 예시3`

In [7]:
def 데이터전처리하기2(examples):
    return 데이터전처리하기1(examples["text"], truncation=True)

In [200]:
전처리된나의데이터 = 나의데이터.map(데이터전처리하기2,batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 569.96 examples/s]


In [209]:
전처리된나의데이터['train'][0]

{'text': 'I prefer making decisions based on logic and objective facts.',
 'label': 0,
 'input_ids': [101,
  1045,
  9544,
  2437,
  6567,
  2241,
  2006,
  7961,
  1998,
  7863,
  8866,
  1012,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

`#`

# 5. 데이터콜렉터 

||주어진자료| $\overset{tokenizer,map}{\Longrightarrow}$ 전처리된자료|$\overset{datacollector}{\Longrightarrow}$더전처리된자료|
|:--:|:--:|:--:|:--:|
|Dict의 Keys |`text`,`label`| `input_ids`, `attention_mask`, `label`| `input_ids`, `attention_mask`, `labels`| 
|자료의형태|텍스트,라벨|숫자화 O, 행렬화 X  | 숫자화 O, 행렬화 O
|`torch.tensor`|- | X | O | 
|미니배치| - | X| O| 
|패딩/동적패딩| - | X | O| 

`-` 데이터콜렉터 사용방법 

In [195]:
데이터콜렉터 = transformers.DataCollatorWithPadding(tokenizer=토크나이저,return_tensors='pt')
데이터콜렉터?

Signature:   데이터콜렉터(features: List[Dict[str, Any]]) -> Dict[str, Any]
Type:        DataCollatorWithPadding
String form:
DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert/distilbert-bas <...> e, special=True),
           }, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')
File:        ~/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/data/data_collator.py
Docstring:  
Data collator that will dynamically pad the inputs received.

Args:
    tokenizer ([`PreTrainedTokenizer`] or [`PreTrainedTokenizerFast`]):
        The tokenizer used for encoding the data.
    padding (`bool`, `str` or [`~utils.PaddingStrategy`], *optional*, defaults to `True`):
        Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
        among:

        - `True` or `'longest'` (default): Pad to the longest sequence in the batch (or no padding if only a single
          sequence is provided

- 당장필요한것: 입력은 `[딕셔너리, 딕셔너리, 딕셔너리, ...]` 의 형태 

In [213]:
데이터콜렉터(
    [
        dict(label=0,input_ids=[101,1045,102],attention_mask=[1,1,1]),
        dict(label=0,input_ids=[101,1045,9544,102],attention_mask=[1,1,1,1]),
    ]
)

{'input_ids': tensor([[ 101, 1045,  102,    0],
        [ 101, 1045, 9544,  102]]), 'attention_mask': tensor([[1, 1, 1, 0],
        [1, 1, 1, 1]]), 'labels': tensor([0, 0])}

In [214]:
인공지능(**데이터콜렉터(
    [
        dict(label=0,input_ids=[101,1045,102],attention_mask=[1,1,1]),
        dict(label=0,input_ids=[101,1045,9544,102],attention_mask=[1,1,1,1]),
    ]
))

SequenceClassifierOutput(loss=tensor(0.7409, grad_fn=<NllLossBackward0>), logits=tensor([[-0.1381, -0.0306],
        [-0.1232, -0.0442]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# 6. 평가하기

`-` `accuracy.compute`의 기능

In [348]:
accuracy = evaluate.load("accuracy")

In [356]:
accuracy.compute(references=[0, 0, 0], predictions=[0, 1, 1])

{'accuracy': 0.3333333333333333}

`-` 함수내용 

In [358]:
# def 평가하기(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     accuracy = evaluate.load("accuracy")
#     return accuracy.compute(predictions=predictions, references=labels)

In [417]:
def 평가하기(eval_pred):
    계산된숫자들_로짓, 실제정답 = eval_pred
    인공지능의예측 = np.argmax(계산된숫자들_로짓, axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=인공지능의예측, references=실제정답)

> `평가하기` 함수는 `eval_dataset`에 적용됨 

# 10. 트레이너 세부지침

In [418]:
트레이너세부지침 = transformers.TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2, # 전체문제세트를 2번 공부하라..
    weight_decay=0.01,
    eval_strategy="epoch", 
    save_strategy="epoch", 
    load_best_model_at_end=True,
    push_to_hub=False,
)

::: {.callout-note} 

### 옵션 설명:

1. **`output_dir="my_awesome_model"`**:
   - 학습된 모델과 관련 파일(예: 체크포인트, 로그 등)이 저장될 디렉토리 경로를 지정합니다.

2. **`learning_rate=2e-5`**:
   - 학습률(learning rate)을 설정합니다. 모델이 가중치를 업데이트할 때 사용하는 스텝 크기로, 작은 값일수록 학습 속도가 느려지지만 안정성이 높습니다.

3. **`per_device_train_batch_size=16`**:
   - **훈련** 시, 각 GPU 또는 CPU 디바이스에서 사용할 배치 크기를 설정합니다. 이 경우 각 디바이스에서 16개의 샘플을 한 번에 처리합니다.

4. **`per_device_eval_batch_size=16`**:
   - **평가** 시, 각 GPU 또는 CPU 디바이스에서 사용할 배치 크기를 설정합니다. 평가 시에는 훈련과 같은 배치 크기를 사용하는 것이 일반적입니다.

5. **`num_train_epochs=2`**:
   - 전체 훈련 데이터셋을 몇 번 반복(에포크)할지 설정합니다. 여기서는 데이터셋 전체를 2번 학습하게 됩니다.

6. **`weight_decay=0.01`**:
   - 가중치 감쇠(weight decay)를 적용하여 모델의 가중치가 지나치게 커지지 않도록 제어합니다. 0.01의 값은 가중치가 조금씩 감소하게 하여 모델의 일반화 성능을 높이는 데 도움을 줄 수 있습니다.

7. **`eval_strategy="epoch"`**:
   - 평가 전략을 설정합니다. 여기서는 `epoch`으로 설정되어, 매 에포크가 끝날 때마다 평가가 진행됩니다.
   - 다른 값으로는 `steps` (일정한 스텝마다 평가) 등이 있습니다.

8. **`save_strategy="epoch"`**:
   - 모델을 언제 저장할지 설정합니다. `epoch`으로 설정되면, 매 에포크가 끝날 때마다 체크포인트를 저장합니다.
   - 다른 값으로는 `steps` (일정한 스텝마다 저장) 등이 있습니다.

9. **`load_best_model_at_end=True`**:
   - 학습이 끝난 후, 가장 성능이 좋았던 체크포인트를 불러옵니다. 평가 지표에 따라 가장 성능이 좋았던 모델을 자동으로 불러와 최종 모델로 사용하게 됩니다.

10. **`push_to_hub=False`**:
    - 모델 학습이 끝난 후 Hugging Face Hub에 모델을 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.
    - `True`로 설정하면 모델과 관련된 파일들이 Hugging Face Hub에 업로드되어 다른 사람들과 공유할 수 있습니다.

:::

`-` lr = 학습률 = 답이 틀렸을 경우 혼내는 정도

- 정확한 느낌은 경사하강법을 이해해야함. 

# 11. 트레이너

`-` 트레이너를 이용한 학습

In [421]:
트레이너 = transformers.Trainer(
    model=인공지능,
    args=트레이너세부지침,
    train_dataset=전처리된데이터['train'],
    eval_dataset=전처리된데이터['test'],
    tokenizer=토크나이저, # 왜 필요하지??
    data_collator=데이터콜렉터,
    compute_metrics=평가하기,
)
트레이너.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.221900,0.199376,0.923000
2,0.145700,0.233206,0.931000


TrainOutput(global_step=3126, training_loss=0.20330691871472223, metrics={'train_runtime': 790.8461, 'train_samples_per_second': 63.223, 'train_steps_per_second': 3.953, 'total_flos': 6556904415524352.0, 'train_loss': 0.20330691871472223, 'epoch': 2.0})

In [423]:
25000 / 16 

1562.5

In [424]:
1563 * 2 

3126

`-` 인공지능이 똑똑해졌을까? 

In [425]:
인공지능.classifier.weight # 숫자들은 바뀌어 있음.. 

Parameter containing:
tensor([[-0.0230,  0.0279,  0.0239,  ...,  0.0085, -0.0062, -0.0143],
        [ 0.0084,  0.0007, -0.0097,  ...,  0.0189, -0.0008,  0.0304]],
       device='cuda:0', requires_grad=True)

In [426]:
인공지능.pre_classifier.weight # 숫자들이 바뀌어 있음.. 

Parameter containing:
tensor([[-0.0133,  0.0017,  0.0303,  ...,  0.0005,  0.0322, -0.0122],
        [-0.0038,  0.0010,  0.0165,  ..., -0.0163, -0.0115, -0.0138],
        [ 0.0033, -0.0210, -0.0019,  ...,  0.0014,  0.0100, -0.0486],
        ...,
        [-0.0043, -0.0345,  0.0094,  ...,  0.0350, -0.0182, -0.0110],
        [ 0.0078, -0.0360, -0.0012,  ...,  0.0021,  0.0170, -0.0120],
        [-0.0162,  0.0133, -0.0184,  ..., -0.0165,  0.0240,  0.0200]],
       device='cuda:0', requires_grad=True)

In [443]:
인공지능.to("cpu") # 인공지능이 지금 GPU에서 학습했거든요.. 다시 CPU로 내려주는 작업이 필요합니다

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

`-` 입력정보에 영화에 대한 부정적 평가를 넣는다면?

In [427]:
입력정보_원시텍스트 = "This movie was a huge disappointment."
정리된숫자_토큰화된자료 = 토크나이저(입력정보_원시텍스트,truncation=True,return_tensors='pt')
정리된숫자_토큰화된자료

{'input_ids': tensor([[  101,  2023,  3185,  2001,  1037,  4121, 10520,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [431]:
인공지능(**정리된숫자_토큰화된자료)

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.2626, -2.1934]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [432]:
계산된숫자_로짓 = 인공지능(**정리된숫자_토큰화된자료).logits.detach().numpy()
계산된숫자_로짓

array([[ 2.262645 , -2.1934297]], dtype=float32)

In [433]:
출력정보_확률 = np.exp(계산된숫자_로짓) / np.exp(계산된숫자_로짓).sum()
출력정보_확률 # 0일확률, 1일확률

array([[0.9885254 , 0.01147464]], dtype=float32)

In [434]:
출력정보_확률.argmax() # 부정적 영화평가에 대한 인공지능의 예측

0

`-` 입력정보에 영화에 대한 긍정적 평가를 넣는다면?

In [436]:
입력정보_원시텍스트 = "This was a masterpiece."
정리된숫자_토큰화된자료 = 토크나이저(입력정보_원시텍스트,truncation=True,return_tensors='pt')
정리된숫자_토큰화된자료

{'input_ids': tensor([[  101,  2023,  2001,  1037, 17743,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [437]:
인공지능(**정리된숫자_토큰화된자료)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.1035,  1.6938]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [438]:
계산된숫자_로짓 = 인공지능(**정리된숫자_토큰화된자료).logits.detach().numpy()
계산된숫자_로짓

array([[-2.103469 ,  1.6938232]], dtype=float32)

In [439]:
출력정보_확률 = np.exp(계산된숫자_로짓) / np.exp(계산된숫자_로짓).sum()
출력정보_확률 # 0일확률, 1일확률

array([[0.0219393, 0.9780607]], dtype=float32)

In [440]:
출력정보_확률.argmax()
계산된숫자_로짓.argmax() # 부정적 영화평가에 대한 인공지능의 예측

1

`-` 우리가 가져야할 생각: 신기하다 X // 노가다 많이 했구나.. O

# 12. 파이프라인

`-` 강인공지능? 

> ref: <https://zdnet.co.kr/view/?no=20160622145838>

In [444]:
강인공지능 = transformers.pipeline("sentiment-analysis", model="my_awesome_model/checkpoint-1563")
print(강인공지능("This movie was a huge disappointment."))
print(강인공지능("This was a masterpiece."))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.9885253310203552}]
[{'label': 'LABEL_1', 'score': 0.978060781955719}]
